In [1]:
import torch
# Vidage RAM GPU
torch.cuda.empty_cache()

## 🧱 CELL 01 — Imports & config

In [2]:
import os, time, math, json, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from pathlib import Path

import os
from pathlib import Path
import sys
# --- Environnement dans le container ---
PROJECT_ROOT = Path("/workspace").resolve()  # ou adapte si différent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [3]:
# === 05_StyleGAN — PixCell-PathoDuet integration (Lite) ===

from p9dg.histo_dataset import HistoDataset




SAMPLES_DIR = (PROJECT_ROOT / "samples").resolve();    SAMPLES_DIR.mkdir(parents=True, exist_ok=True)
CKPT_DIR    = (PROJECT_ROOT / "checkpoints").resolve();CKPT_DIR.mkdir(parents=True, exist_ok=True)
ARTI_DIR    = (PROJECT_ROOT / "artifacts_pixcell").resolve(); ARTI_DIR.mkdir(parents=True, exist_ok=True)

# Si déjà définis ailleurs, garde tes valeurs :
Z_DIM      = globals().get("Z_DIM", 512)
BATCH_SIZE = globals().get("BATCH_SIZE", 8)
USE_AMP    = globals().get("USE_AMP", True)

## 🧱 CELL 2 — PathoDuet: import robuste + instance + freeze + encode_image

In [4]:
# === CELL 2 — PathoDuet: import robuste + instance + freeze + encode_image (adapté ckpt HE) ===
import re, os, torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Si tu as DEJA un objet ou une classe dans le notebook, on le réutilise tel quel
if 'pathoduet' in globals():
    _pd_backend = 'existing_object'
    _pd_model = pathoduet
elif 'PathoDuetModel' in globals():
    _pd_backend = 'existing_class'
    _pd_model = PathoDuetModel.from_pretrained("pathoduet-base")
else:
    # 2) Chargement LOCAL conforme à ton snippet (timm DeiT-B/16 + checkpoint_HE)
    _pd_backend = 'local_deit_b16_ckptHE'
    try:
        import timm
    except Exception as e:
        raise RuntimeError("Installe 'timm' (pip install timm)") from e

    assert 'PROJECT_ROOT' in globals(), "PROJECT_ROOT non défini."
    CKPT_PATH = PROJECT_ROOT / "models/checkpoint_HE.pth"
    assert CKPT_PATH.exists(), f"Checkpoint introuvable: {CKPT_PATH}"

    def load_state_dict_safely(path):
        sd_raw = None
        try:
            obj = torch.load(path, map_location="cpu", weights_only=True)
            if isinstance(obj, dict) and all(hasattr(v, "shape") for v in obj.values()):
                sd_raw = obj
            elif isinstance(obj, dict):
                for k in ("state_dict","model","module"):
                    if k in obj and isinstance(obj[k], dict) and all(hasattr(v, "shape") for v in obj[k].values()):
                        sd_raw = obj[k]; break
        except TypeError:
            obj = torch.load(path, map_location="cpu")
            if isinstance(obj, dict) and all(hasattr(v, "shape") for v in obj.values()):
                sd_raw = obj
            elif isinstance(obj, dict):
                for k in ("state_dict","model","module"):
                    if k in obj and isinstance(obj[k], dict) and all(hasattr(v, "shape") for v in obj[k].values()):
                        sd_raw = obj[k]; break
        if sd_raw is None:
            raise RuntimeError("state_dict non trouvé dans le ckpt")

        clean_sd = {re.sub(r"^(module\.|model\.)","",k): v for k, v in sd_raw.items()}
        # retirer distillation + réajuster pos_embed 198→197
        clean_sd.pop("dist_token", None)
        for k in list(clean_sd.keys()):
            if k.startswith("head_dist."): clean_sd.pop(k)
        if "pos_embed" in clean_sd and clean_sd["pos_embed"].shape[1] == 198:
            pos = clean_sd["pos_embed"]
            clean_sd["pos_embed"] = torch.cat([pos[:, :1, :], pos[:, 2:, :]], dim=1)
        for k in list(clean_sd.keys()):
            if k.startswith("head."): clean_sd.pop(k)
        return clean_sd

    state_dict = load_state_dict_safely(CKPT_PATH)
    backbone = timm.create_model(
        "deit_base_patch16_224",
        pretrained=False,
        num_classes=0,    # vecteur feature directement
        img_size=224,
        global_pool="avg" # GAP → (B, 768)
    )
    missing, unexpected = backbone.load_state_dict(state_dict, strict=False)
    if missing or unexpected:
        print(f"[PathoDuet] missing: {missing[:5]}  | unexpected: {unexpected[:5]}")

    for p in backbone.parameters(): p.requires_grad_(False)
    backbone.eval()

    class _PathoDuetExtractor(nn.Module):
        """
        Attend x dans [-1,1]. Redimensionne en 224x224. Retourne embedding (B, D).
        """
        def __init__(self, bb, in_size=224):
            super().__init__()
            self.bb = bb
            self.in_size = in_size
            self.dim = None

        @torch.no_grad()
        def forward(self, x_m11: torch.Tensor) -> torch.Tensor:
            # [-1,1] -> [0,1]
            x01 = (x_m11.clamp(-1,1) + 1)*0.5
            if x01.shape[-2:] != (self.in_size, self.in_size):
                x01 = F.interpolate(x01, size=(self.in_size,self.in_size),
                                    mode="bilinear", align_corners=False)
            feats = self.bb(x01)               # (B, D) attendu (D=768 sur DeiT-B/16)
            if self.dim is None: self.dim = feats.shape[-1]
            return feats

        @torch.no_grad()
        def encode_image(self, x: torch.Tensor) -> torch.Tensor:
            return self.forward(x)

    _pd_model = _PathoDuetExtractor(backbone)

# 3) Normaliser l’API: garantir pathoduet.encode_image(x) -> (B, EMB_DIM) L2-normalisé
class _EncodeImageWrapper(nn.Module):
    def __init__(self, base):
        super().__init__()
        self.base = base
        self.dim = None  # sera inféré au 1er appel
        self.base.eval()
        for p in self.base.parameters():
            p.requires_grad_(False)

        self._has_encode = hasattr(self.base, "encode_image") and callable(getattr(self.base, "encode_image"))
        self._has_forward = hasattr(self.base, "forward") and callable(getattr(self.base, "forward"))

    @torch.no_grad()
    def encode_image(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B,3,H,W) float dans [-1,1] (conforme à ton pipeline GAN), ou [0,1] si déjà remappé.
        return: (B, EMB_DIM) L2-normalisé
        """
        self.base.eval()
        if self._has_encode:
            feats = self.base.encode_image(x)
        else:
            out_raw = self.base(x) if self._has_forward else self.base.forward(x)
            if isinstance(out_raw, torch.Tensor):
                feats = out_raw
            elif hasattr(out_raw, "last_hidden_state"):
                feats = out_raw.last_hidden_state.mean(dim=1)
            elif hasattr(out_raw, "pooler_output"):
                feats = out_raw.pooler_output
            else:
                tensors = [v for v in out_raw.__dict__.values() if isinstance(v, torch.Tensor)]
                if not tensors:
                    raise RuntimeError("Impossible d'extraire des features du modèle PathoDuet (structure inconnue).")
                feats = tensors[0]
            if feats.ndim == 3:   # (B, T, D)
                feats = feats.mean(dim=1)
            elif feats.ndim > 2:  # (B, C, H, W) -> GAP
                feats = F.adaptive_avg_pool2d(feats, 1).flatten(1)

        if self.dim is None:
            self.dim = feats.shape[-1]
        return F.normalize(feats, dim=-1)

# 4) Exposer l’instance normalisée `pathoduet` et EMB_DIM (on force une passe dummy pour fixer dim)
pathoduet = _EncodeImageWrapper(_pd_model).to(DEVICE).eval()
with torch.no_grad():
    _dummy = torch.zeros(1,3,256,256, device=DEVICE)  # taille quelconque
    _ = pathoduet.encode_image(_dummy)
EMB_DIM = pathoduet.dim or 768

print(f"[PathoDuet] backend={_pd_backend}  | EMB_DIM={EMB_DIM}  | device={DEVICE}")


[PathoDuet] missing: []  | unexpected: ['pretext_token']
[PathoDuet] backend=local_deit_b16_ckptHE  | EMB_DIM=768  | device=cuda


## 🧱 CELL 3 — Cache d'embeddings PathoDuet (train/val) + prototypes μ_k

In [5]:
# === CELL 3 bis — Datasets (GAN + Embeddings PathoDuet) + ÉCHANTILLONNAGE + cache + μ_k (train only) ===
import os, numpy as np, torch
from pathlib import Path
from collections import defaultdict
from torch.utils.data import DataLoader
from p9dg.histo_dataset import HistoDataset

# ---- Chemins & config (conteneur) ----
DATA_ROOT   = Path(os.getenv("DATA_ROOT", "/workspace/data")).resolve()
CONFIG_DIR  = Path(os.getenv("CONFIG_DIR", str(PROJECT_ROOT))).resolve()
THR_PATH    = CONFIG_DIR / "seuils_par_classe.json"
assert THR_PATH.exists(), f"Fichier de seuils introuvable: {THR_PATH}"

# ---- Réglages (reprend ceux définis plus haut si présents) ----
IMAGE_SIZE        = globals().get("IMAGE_SIZE", 128)
PIXEL_RANGE       = globals().get("PIXEL_RANGE", "-1_1")
VAHADANE_ENABLE   = globals().get("VAHADANE_ENABLE", True)
SAMPLES_PER_CLASS = globals().get("SAMPLES_PER_CLASS", None)
SELECTED_CLASSES  = globals().get("SELECTED_CLASSES", None)
NUM_WORKERS       = globals().get("NUM_WORKERS", 0)
DEVICE            = "cuda" if torch.cuda.is_available() else "cpu"

# ---- Contrôle de l'effort d'extraction embeddings ----
EMB_MAX_PER_CLASS = globals().get("EMB_MAX_PER_CLASS", 500)  # ← budget par classe (ex: 500)
BATCH_EMB         = globals().get("BATCH_EMB", 128)
EXTRACT_VAL_EMB   = globals().get("EXTRACT_VAL_EMB", False)  # ← par défaut, on N'EXTRAIT PAS sur val

# ---- Dossiers artefacts ----
ARTI_DIR = (PROJECT_ROOT / "artifacts_pixcell").resolve(); ARTI_DIR.mkdir(parents=True, exist_ok=True)
EMB_DIR  = (ARTI_DIR / "emb_cache").resolve(); EMB_DIR.mkdir(parents=True, exist_ok=True)

# ============================
#  A) Dataset principal GAN (inchangé)
# ============================
ds_gan = HistoDataset(
    root_data=str(DATA_ROOT),
    split="train",
    output_size=IMAGE_SIZE,
    pixel_range=PIXEL_RANGE,
    balance_per_class=True,
    samples_per_class_per_epoch=SAMPLES_PER_CLASS,
    vahadane_enable=VAHADANE_ENABLE,
    vahadane_device=DEVICE,
    thresholds_json_path=str(THR_PATH),
    return_labels=True,
    classes=SELECTED_CLASSES,
)
print(f"[ds_gan] {len(ds_gan)} items | classes: {ds_gan.class_counts()}")

# ============================
#  B) Dataset embeddings (TRAIN uniquement, en -1_1)
# ============================
ds_tr_emb = HistoDuet = HistoDataset(
    root_data=str(DATA_ROOT),
    split="train",
    output_size=IMAGE_SIZE,
    pixel_range="-1_1",             # PathoDuet attend [-1,1]
    balance_per_class=True,         # donne un flux équilibré
    samples_per_class_per_epoch=None,  # on maîtrise l'échantillonnage nous-mêmes ci-dessous
    vahadane_enable=VAHADANE_ENABLE,
    vahadane_device=DEVICE,
    thresholds_json_path=str(THR_PATH),
    return_labels=True,
    classes=SELECTED_CLASSES,
)
dl_tr_emb = DataLoader(ds_tr_emb, batch_size=BATCH_EMB, shuffle=True, num_workers=NUM_WORKERS, pin_memory=False)
class_to_idx = ds_tr_emb.class_to_idx
idx_to_class = ds_tr_emb.idx_to_class
num_classes  = len(class_to_idx)
print(f"[embeddings/train] classes ({num_classes}): {class_to_idx}")

# ============================
#  C) Extraction + cache (avec PLAFOND par classe)
# ============================
assert 'pathoduet' in globals() and 'EMB_DIM' in globals(), "Exécute d'abord la cellule PathoDuet (Cell 2)."

def cache_embeddings_capped(dl, split_name: str, max_per_class: int):
    fpath = EMB_DIR / f"{split_name}_feats_np{max_per_class}.npy"
    lpath = EMB_DIR / f"{split_name}_labels_np{max_per_class}.npy"
    if fpath.exists() and lpath.exists():
        feats = torch.from_numpy(np.load(fpath))
        labs  = torch.from_numpy(np.load(lpath))
        print(f"[cache] {split_name}: chargés {tuple(feats.shape)} depuis {fpath.name}")
        return feats, labs

    counts = defaultdict(int)
    all_f, all_y = [], []
    total_target = max_per_class * num_classes

    pathoduet.eval()
    with torch.no_grad():
        for xb, yb, _ in dl:
            # stop si on a atteint le budget global
            if sum(counts.values()) >= total_target:
                break

            xb = xb.to(DEVICE, non_blocking=True)
            fb = pathoduet.encode_image(xb)  # (B, EMB_DIM), L2-normalisé

            # filtrage par classe pour respecter max_per_class
            for f_row, y in zip(fb.cpu(), yb.cpu()):
                if counts[int(y)] < max_per_class:
                    all_f.append(f_row.unsqueeze(0))
                    all_y.append(torch.tensor([int(y)]))
                    counts[int(y)] += 1
                # early stop si on a rempli toutes les classes
                if len(counts) == num_classes and min(counts.values()) >= max_per_class:
                    break

    if not all_f:
        raise RuntimeError("Aucun embedding collecté — vérifie les chemins et le budget EMB_MAX_PER_CLASS.")
    F = torch.cat(all_f, 0)  # (K, EMB_DIM)
    Y = torch.cat(all_y, 0).view(-1)  # (K,)
    np.save(fpath, F.numpy()); np.save(lpath, Y.numpy())
    print(f"[cache] {split_name}: sauvegardés {tuple(F.shape)} → {fpath.name}")
    print(f"[budget par classe] ", {idx_to_class[k]: c for k,c in counts.items()})
    return F, Y

Ftr, Ytr = cache_embeddings_capped(dl_tr_emb, "train", EMB_MAX_PER_CLASS)

# (Optionnel) banque embeddings val — par défaut désactivée pour éviter tout mélange
if EXTRACT_VAL_EMB:
    ds_va_emb = HistoDataset(
        root_data=str(DATA_ROOT),
        split="val",
        output_size=IMAGE_SIZE,
        pixel_range="-1_1",
        balance_per_class=True,
        samples_per_class_per_epoch=None,
        vahadane_enable=VAHADANE_ENABLE,
        vahadane_device=DEVICE,
        thresholds_json_path=str(THR_PATH),
        return_labels=True,
        classes=SELECTED_CLASSES,
    )
    dl_va_emb = DataLoader(ds_va_emb, batch_size=BATCH_EMB, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    Fva, Yva = cache_embeddings_capped(dl_va_emb, "val", EMB_MAX_PER_CLASS)
else:
    Fva, Yva = None, None

# ============================
#  D) Prototypes μ_k (sur train)
# ============================
@torch.no_grad()
def class_prototypes(F: torch.Tensor, Y: torch.Tensor):
    protos = {}
    for k in torch.unique(Y):
        m = (Y == int(k))
        protos[int(k)] = F[m].mean(0)   # (EMB_DIM,)
    return protos

PROTOS = class_prototypes(Ftr, Ytr)
print("μ_k prêts →", {idx_to_class[k]: tuple(v.shape) for k, v in list(PROTOS.items())[:min(3, len(PROTOS))]})


🎨 Référence Vahadane fixée : TUM-RQEVGAED.tif
🎨 Référence Vahadane auto: TUM-RQEVGAED.tif
✅ Seuils par classe chargés depuis : /workspace/configs/seuils_par_classe.json
⚖️ Échantillonnage équilibré auto (8763 images / classe, min du dataset).
[ds_gan] 78867 items | classes: {'ADI': 10407, 'BACK': 10566, 'DEB': 11512, 'LYM': 11557, 'MUC': 8896, 'MUS': 13536, 'NORM': 8763, 'STR': 10446, 'TUM': 14317}
🎨 Référence Vahadane fixée : TUM-RQEVGAED.tif
🎨 Référence Vahadane auto: TUM-RQEVGAED.tif
✅ Seuils par classe chargés depuis : /workspace/configs/seuils_par_classe.json
⚖️ Échantillonnage équilibré auto (8763 images / classe, min du dataset).
[embeddings/train] classes (9): {'ADI': 0, 'BACK': 1, 'DEB': 2, 'LYM': 3, 'MUC': 4, 'MUS': 5, 'NORM': 6, 'STR': 7, 'TUM': 8}
[cache] train: chargés (4500, 768) depuis train_feats_np500.npy
μ_k prêts → {'ADI': (768,), 'BACK': (768,), 'DEB': (768,)}


## 🧱 CELL 4 — Adapter (EMB_DIM → C_DIM) + utilitaires μ_k → c 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

assert 'EMB_DIM' in globals() and 'PROTOS' in globals(), "Exécute d'abord Cell 2 (PathoDuet) et Cell 3 (μ_k)."
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Dimension de condition côté GAN (modifiable)
C_DIM = globals().get("C_DIM", 128)

class PixCellAdapter(nn.Module):
    """
    Projette l'embedding PathoDuet (EMB_DIM) vers l'espace de condition (C_DIM).
    Petit MLP suffisant pour le POC.
    """
    def __init__(self, in_dim: int = EMB_DIM, out_dim: int = C_DIM):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256), nn.SiLU(),
            nn.Linear(256, out_dim)
        )
    def forward(self, e: torch.Tensor) -> torch.Tensor:
        # e: (B, EMB_DIM) — déjà L2-normalisé par PathoDuet
        return self.net(e)

adapter = PixCellAdapter().to(DEVICE).eval()  # on l'entraînera plus tard (Cellule entraînement "lite")

@torch.no_grad()
def proto_tensor(k: int) -> torch.Tensor:
    """
    Retourne le prototype μ_k (EMB_DIM,) sur le bon device.
    """
    e = PROTOS[int(k)].to(DEVICE)
    return e

@torch.no_grad()
def cond_from_proto(labels: torch.Tensor, noise_std: float = 0.0, mixup_alpha: float | None = None) -> torch.Tensor:
    """
    Convertit une liste d'indices de classe → vecteurs de condition c via:
      1) récupération des prototypes μ_k
      2) option: bruit gaussien dans l'espace embedding
      3) projection via l'Adapter (EMB_DIM → C_DIM)

    Args:
        labels: (B,) tensor long
        noise_std: std du bruit dans l'espace PathoDuet (petite diversité intra-classe)
        mixup_alpha: si défini, mélange de deux prototypes (Beta(alpha, alpha)) pour diversifier

    Returns:
        c: (B, C_DIM)
    """
    B = labels.numel()
    E = []
    for k in labels.tolist():
        e = proto_tensor(k)
        if mixup_alpha is not None:
            # mixup entre μ_k et un autre prototype aléatoire
            k2 = int(torch.randint(low=0, high=len(PROTOS), size=(1,)).item())
            e2 = proto_tensor(k2)
            lam = torch.distributions.Beta(mixup_alpha, mixup_alpha).sample().to(DEVICE)
            e = lam * e + (1 - lam) * e2
            e = F.normalize(e, dim=0)
        if noise_std > 0:
            e = e + noise_std * torch.randn_like(e)
            e = F.normalize(e, dim=0)
        E.append(e)
    E = torch.stack(E, 0)       # (B, EMB_DIM)
    c = adapter(E)              # (B, C_DIM)
    return c

# --- Sanity check rapide ---
with torch.no_grad():
    dummy_labels = torch.tensor([0, 1, 2], device=DEVICE)  # adapte si ton mapping débute ailleurs
    c_test = cond_from_proto(dummy_labels, noise_std=0.0)
print(f"[Adapter] C_DIM={C_DIM} | c_test shape={tuple(c_test.shape)}")

[Adapter] C_DIM=128 | c_test shape=(3, 128)


## 🧱 CELL 5 — Wrappers conditionnels pour G et D (z' = z + M(c), projection trick côté D) ===

In [7]:
# === CELL GD-1 — Générateur & Discriminateur jouets (compatibles [-1,1], logits (B,)) ===
import math, torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Valeurs par défaut si non définies
IMAGE_SIZE = globals().get("IMAGE_SIZE", 256)
Z_DIM      = globals().get("Z_DIM", 512)

assert IMAGE_SIZE in (64, 128, 256), "Ce G/D jouet supporte 64, 128 ou 256 pour simplifier."

# --------- Utils ---------
def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

def weights_init_leaky(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.kaiming_normal_(m.weight, a=0.2, mode="fan_in", nonlinearity="leaky_relu")
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# --------- Blocks ---------
class UpBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.norm = nn.GroupNorm(8, out_ch)
        self.act  = nn.SiLU()
    def forward(self, x):
        x = F.interpolate(x, scale_factor=2, mode="nearest")
        x = self.conv(x)
        x = self.norm(x)
        return self.act(x)

class DownBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 4, stride=2, padding=1)
        self.act  = nn.LeakyReLU(0.2, inplace=True)
    def forward(self, x):
        return self.act(self.conv(x))

# --------- Générateur ---------
class ToyGenerator(nn.Module):
    """
    z -> linear -> (C0, 4, 4) -> upsample xN -> conv -> Tanh
    Sortie en [-1,1].
    """
    def __init__(self, z_dim=Z_DIM, img_size=IMAGE_SIZE, base=512):
        super().__init__()
        self.z_dim   = z_dim
        self.img_sz  = img_size
        self.map     = nn.Linear(z_dim, base*4*4)

        # Échelle de canaux selon image size
        # 4 -> 8 -> 16 -> 32 -> 64 -> 128 -> 256
        chs = [base, 256, 128, 64, 32, 16, 8]
        depth_needed = {64: 4, 128: 5, 256: 6}[img_size]  # nb d'upsamples à partir de 4x4
        blocks = []
        in_c = chs[0]
        for i in range(depth_needed):
            out_c = chs[i+1]
            blocks.append(UpBlock(in_c, out_c))
            in_c = out_c
        self.ups = nn.Sequential(*blocks)
        self.to_rgb = nn.Conv2d(in_c, 3, 3, padding=1)

    def forward(self, z):
        b = z.size(0)
        x = self.map(z).view(b, -1, 4, 4)
        x = self.ups(x)
        x = torch.tanh(self.to_rgb(x))   # [-1, 1]
        return x

# --------- Discriminateur ---------
class ToyDiscriminator(nn.Module):
    """
    PatchGAN-like global : downsample jusqu'à 4x4, puis FC -> 1
    Sortie: logit (B,)
    """
    def __init__(self, img_size=IMAGE_SIZE, base=64):
        super().__init__()
        # 3 -> 64 -> 128 -> 256 -> 512 -> 512 -> ...
        chans = [3, base, base*2, base*4, base*8, base*8]
        blocks = []
        for i in range(len(chans)-1):
            blocks.append(DownBlock(chans[i], chans[i+1]))
        self.down = nn.Sequential(*blocks)

        # Taille spatiale après downsamples 2^n
        # On applique n downsamples pour atteindre 4x4
        n_down = int(math.log2(img_size//4))
        # La séquence ci-dessus fait len(chans)-1 downsamples ; c'est suffisant pour 256/128/64
        # On adapte le pooling final au spatial restant
        self.pool = nn.AdaptiveAvgPool2d((4,4))
        self.fc   = nn.Linear(chans[-1]*4*4, 1)

    def forward(self, x):
        h = self.down(x)
        h = self.pool(h)
        h = h.flatten(1)
        logit = self.fc(h).squeeze(1)  # (B,)
        return logit

# --------- Instanciation + init ---------
G = ToyGenerator(z_dim=Z_DIM, img_size=IMAGE_SIZE).to(DEVICE)
D = ToyDiscriminator(img_size=IMAGE_SIZE).to(DEVICE)
G.apply(weights_init_leaky)
D.apply(weights_init_leaky)

print(f"G params: {count_params(G):,} | D params: {count_params(D):,}")


G params: 5,775,747 | D params: 6,959,553


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

assert 'G' in globals() and 'D' in globals(), "Instancie d'abord tes modèles G et D."
assert 'Z_DIM' in globals() and 'C_DIM' in globals(), "Z_DIM et C_DIM doivent être définis."
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ----- Générateur conditionné : z' = z + M(c) -----
class CondGenerator(nn.Module):
    def __init__(self, G_base: nn.Module, z_dim: int, c_dim: int):
        super().__init__()
        self.G = G_base
        self.z_dim = z_dim
        self.c_dim = c_dim
        # petit mapper M: c -> R^{z_dim}
        self.map = nn.Sequential(
            nn.Linear(c_dim, max(128, z_dim)), nn.SiLU(),
            nn.Linear(max(128, z_dim), z_dim)
        )
    def forward(self, z: torch.Tensor, c: torch.Tensor, **kwargs):
        # z: (B, Z_DIM), c: (B, C_DIM)
        z_mod = z + self.map(c)
        return self.G(z_mod, **kwargs)

# ----- Discriminateur conditionné : projection trick -----
class CondDiscriminator(nn.Module):
    """
    logits = D(x) + <h(x), Wc>
    Si D ne fournit pas de features, on fabrique h(x) via une petite tête hproj().
    """
    def __init__(self, D_base: nn.Module, c_dim: int, feat_dim: int = 256):
        super().__init__()
        self.D = D_base
        self.c_dim = c_dim
        self.feat_dim = feat_dim
        # tête pour extraire h(x) à partir de l'image (faible coût, non invasive)
        self.hproj = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=1, bias=False),
            nn.SiLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32, feat_dim),
            nn.SiLU()
        )
        # projection conditionnelle W c
        self.proj = nn.Linear(c_dim, feat_dim, bias=False)

    def forward(self, x: torch.Tensor, c: torch.Tensor):
        base = self.D(x)                # (B,) ou (B,1)
        if base.ndim == 2:
            base = base.squeeze(1)
        h = self.hproj(x)               # (B, feat_dim)
        wc = self.proj(c)               # (B, feat_dim)
        return base + (h * wc).sum(dim=1)

# ----- Instanciation -----
Gc = CondGenerator(G, z_dim=Z_DIM, c_dim=C_DIM).to(DEVICE)
Dc = CondDiscriminator(D, c_dim=C_DIM, feat_dim=256).to(DEVICE)

# ----- Sanity check rapide -----
with torch.no_grad():
    B = 4
    z = torch.randn(B, Z_DIM, device=DEVICE)
    x_fake = Gc(z, torch.randn(B, C_DIM, device=DEVICE)).clamp(-1, 1)  # supposé [-1,1] dans ton pipeline
    _ = Dc(x_fake, torch.randn(B, C_DIM, device=DEVICE))
print(f"[Wrappers] OK → Gc(z,c)→{tuple(x_fake.shape)} ; Dc(x,c)→(B,)")

# ----- (option) Politiques de gel pour l'entraînement 'lite' (on gèle presque tout sauf les petites têtes) -----
def set_requires_grad(module, flag: bool):
    for p in module.parameters():
        p.requires_grad_(flag)

# Gèle G et D d'origine
set_requires_grad(G, False)
set_requires_grad(D, False)

# Autorise l'entraînement des parties ajoutées (mapper de Gc, hproj & proj de Dc)
for p in Gc.map.parameters(): p.requires_grad_(True)
for p in Dc.hproj.parameters(): p.requires_grad_(True)
for p in Dc.proj.parameters(): p.requires_grad_(True)

print("[Freeze] G & D gelés ; têtes conditionnelles (Gc.map, Dc.hproj, Dc.proj) entraînables.")

[Wrappers] OK → Gc(z,c)→(4, 3, 128, 128) ; Dc(x,c)→(B,)
[Freeze] G & D gelés ; têtes conditionnelles (Gc.map, Dc.hproj, Dc.proj) entraînables.


### 🧪 Cellule test — smoke test direct + (si présent) test avec Gc/Dc

In [9]:
# === CELL GD-2 — Smoke tests (brut + via wrappers si déjà définis) ===
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
B = 4

with torch.no_grad():
    z = torch.randn(B, Z_DIM, device=DEVICE)
    x_fake = G(z).clamp(-1,1)
    d_out  = D(x_fake)
print(f"[RAW] G(z)->{tuple(x_fake.shape)} in [{x_fake.min().item():.2f},{x_fake.max().item():.2f}] | D(x)->{tuple(d_out.shape)}")

# Test via wrappers (Cell 5) si déjà définis
if 'Gc' in globals() and 'Dc' in globals() and 'cond_from_proto' in globals():
    with torch.no_grad():
        # fabrique un c bidon si pas de PROTOS (sinon utilise un vrai)
        if 'PROTOS' in globals() and len(PROTOS)>0:
            labels = torch.tensor([0]*B, device=DEVICE)  # 0 = première classe du mapping
            c = cond_from_proto(labels, noise_std=0.0)
        else:
            C_DIM = globals().get("C_DIM", 128)
            c = torch.randn(B, C_DIM, device=DEVICE)

        z = torch.randn(B, Z_DIM, device=DEVICE)
        x_fake2 = Gc(z, c).clamp(-1,1)
        d_out2  = Dc(x_fake2, c)
    print(f"[WRAPPED] Gc(z,c)->{tuple(x_fake2.shape)} | Dc(x,c)->{tuple(d_out2.shape)}")
else:
    print("[WRAPPED] Wrappers Gc/Dc non trouvés (exécute la Cellule 5 pour les créer).")


[RAW] G(z)->(4, 3, 128, 128) in [-1.00,1.00] | D(x)->(4,)
[WRAPPED] Gc(z,c)->(4, 3, 128, 128) | Dc(x,c)->(4,)


## 🧱 CELL 6 — Entraînement "Lite" (Adapter + têtes conditionnelles)

In [10]:
# === CELL 6-LOGGER — TensorBoard + CSV (un par run) + tracés Matplotlib ===
import os, time, json, math, pandas as pd
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

# Dossiers
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", ".")).resolve()
ARTI_DIR     = (PROJECT_ROOT / "artifacts_pixcell").resolve(); ARTI_DIR.mkdir(parents=True, exist_ok=True)
RUNS_DIR     = (PROJECT_ROOT / "runs_pixcell").resolve();       RUNS_DIR.mkdir(parents=True, exist_ok=True)

# Identifiant de run (un CSV par run)
RUN_ID   = datetime.now().strftime("%Y%m%d_%H%M%S")
TB_DIR   = RUNS_DIR / RUN_ID
CSV_PATH = ARTI_DIR / f"train_log_{RUN_ID}.csv"
print(f"🧪 RUN_ID={RUN_ID}\n📁 TensorBoard: {TB_DIR}\n📄 CSV: {CSV_PATH.name}")

writer = SummaryWriter(log_dir=str(TB_DIR))
_history = []  # liste de dicts

def log_metrics(step:int,
                d_loss:float=None, g_loss:float=None,
                r1:float=None,
                ada_p:float=None, ada_acc:float=None,
                lrD:float=None, lrG:float=None, lrA:float=None):
    """Enregistre dans TensorBoard + mémoire; le CSV est (ré)écrit périodiquement."""
    row = {"step": step, "time_s": time.time()}
    if d_loss is not None: row["loss_D"] = float(d_loss); writer.add_scalar("loss/D", d_loss, step)
    if g_loss is not None: row["loss_G"] = float(g_loss); writer.add_scalar("loss/G", g_loss, step)
    if r1     is not None: row["loss_R1"] = float(r1);    writer.add_scalar("reg/R1", r1, step)
    if ada_p  is not None: row["ADA_p"]   = float(ada_p); writer.add_scalar("ADA/p", ada_p, step)
    if ada_acc is not None:row["ADA_acc"] = float(ada_acc);writer.add_scalar("ADA/acc_ema", ada_acc, step)
    if lrD is not None:    row["lr_D"]    = float(lrD);   writer.add_scalar("lr/D", lrD, step)
    if lrG is not None:    row["lr_G"]    = float(lrG);   writer.add_scalar("lr/G", lrG, step)
    if lrA is not None:    row["lr_A"]    = float(lrA);   writer.add_scalar("lr/A", lrA, step)

    _history.append(row)

def flush_history(to_csv_every:int=200, force:bool=False):
    """Écrit/actualise le CSV unique de ce run toutes les N itérations (ou forcé en fin de run)."""
    if (not force) and (len(_history) % to_csv_every): 
        return
    if not _history:
        return
    df = pd.DataFrame(_history).sort_values("step")
    df.to_csv(CSV_PATH, index=False)

def plot_training():
    """Trace rapide des courbes depuis la mémoire (_history)."""
    if not _history:
        print("Pas encore d'historique.")
        return
    df = pd.DataFrame(_history).sort_values("step")
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    # Losses
    axes[0,0].plot(df["step"], df.get("loss_D", pd.Series()), label="D")
    axes[0,0].plot(df["step"], df.get("loss_G", pd.Series()), label="G")
    axes[0,0].set_title("Losses (D / G)"); axes[0,0].legend(); axes[0,0].grid(True, alpha=0.3)

    # R1
    if "loss_R1" in df:
        axes[0,1].plot(df["step"], df["loss_R1"])
        axes[0,1].set_title("R1 penalty"); axes[0,1].grid(True, alpha=0.3)
    else:
        axes[0,1].axis("off")

    # ADA
    if "ADA_p" in df or "ADA_acc" in df:
        if "ADA_p" in df:   axes[1,0].plot(df["step"], df["ADA_p"],  label="p")
        if "ADA_acc" in df: axes[1,0].plot(df["step"], df["ADA_acc"], label="acc_ema")
        axes[1,0].set_title("ADA"); axes[1,0].legend(); axes[1,0].grid(True, alpha=0.3)
    else:
        axes[1,0].axis("off")

    # LRs
    has_lr = any(k in df for k in ("lr_D","lr_G","lr_A"))
    if has_lr:
        if "lr_D" in df: axes[1,1].plot(df["step"], df["lr_D"], label="lr_D")
        if "lr_G" in df: axes[1,1].plot(df["step"], df["lr_G"], label="lr_G")
        if "lr_A" in df: axes[1,1].plot(df["step"], df["lr_A"], label="lr_A")
        axes[1,1].set_title("Learning Rates"); axes[1,1].legend(); axes[1,1].grid(True, alpha=0.3)
    else:
        axes[1,1].axis("off")

    plt.tight_layout(); plt.show()
    print(f"➡️ Ouvre TensorBoard pour plus de détails : tensorboard --logdir {RUNS_DIR}")


INFO:matplotlib.font_manager:generated new fontManager


🧪 RUN_ID=20251030_224524
📁 TensorBoard: /workspace/runs_pixcell/20251030_224524
📄 CSV: train_log_20251030_224524.csv


In [11]:
# === CELL 6 — FINAL (Lite training stabilisé) ===
import time, torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from pathlib import Path

assert {'Gc','Dc','G','D','adapter','cond_from_proto','ds_gan'} <= set(globals()), \
    "Pré-requis manquants (exécute les Cellules 3–5)."

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT", ".")).resolve()
SAMPLES_DIR  = (PROJECT_ROOT / "samples").resolve(); SAMPLES_DIR.mkdir(parents=True, exist_ok=True)

# --------------------------
# 1) z' = z + α*M(c) (résiduel stable)
# --------------------------
RES_SCALE = 0.1
def _forward_gc_residual(self, z, c, **kw):
    return self.G(z + RES_SCALE * self.map(c), **kw)
Gc.forward = _forward_gc_residual.__get__(Gc, type(Gc))

# --------------------------
# 2) Spectral norm sur Dc.hproj / Dc.proj
# --------------------------
def _apply_sn(module):
    for m in module.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            try: torch.nn.utils.spectral_norm(m)
            except Exception: pass
_apply_sn(Dc.hproj); _apply_sn(Dc.proj)

# --------------------------
# 3) Politique de gel (léger dégel utile)
# --------------------------
def set_requires_grad(m, flag):
    for p in m.parameters(): p.requires_grad_(flag)

set_requires_grad(G, False); set_requires_grad(D, False)          # gèle les bases
set_requires_grad(Gc.map, True); set_requires_grad(Dc.hproj, True); set_requires_grad(Dc.proj, True)
if hasattr(G, "ups") and len(G.ups)>0: set_requires_grad(G.ups[-1], True)
if hasattr(G, "to_rgb"):                set_requires_grad(G.to_rgb, True)
if hasattr(D, "fc"):                    set_requires_grad(D.fc, True)

# --------------------------
# 4) DataLoader (conforme à ton contexte)
# --------------------------
BATCH_SIZE  = globals().get("BATCH_SIZE", 16)
Z_DIM       = globals().get("Z_DIM", 512)
C_DIM       = globals().get("C_DIM", 128)
USE_AMP     = globals().get("USE_AMP", True)
R1_EVERY    = globals().get("R1_EVERY", 16)
R1_GAMMA    = globals().get("R1_GAMMA", 10.0)
CLIP_D      = globals().get("CLIP_D", 5.0)
CLIP_G      = globals().get("CLIP_G", 10.0)
STEPS_LITE  = 20_00 # nombre total d'itérations
LOG_EVERY   = globals().get("LOG_EVERY", 200)
SAMPLE_EVERY= globals().get("SAMPLE_EVERY", 500) # echantillons périodiques

train_dl_gan = DataLoader(ds_gan, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=0, pin_memory=False, drop_last=True)

# --------------------------
# 5) Pertes & optims (LRs sages)
# --------------------------
def d_logistic_loss(real_pred, fake_pred):
    return F.softplus(-real_pred).mean() + F.softplus(fake_pred).mean()
def g_nonsat_loss(fake_pred):
    return F.softplus(-fake_pred).mean()

params_D = list(Dc.hproj.parameters()) + list(Dc.proj.parameters())
if hasattr(D, "fc"): params_D += list(D.fc.parameters())
params_G = list(Gc.map.parameters())
if hasattr(G, "ups") and len(G.ups)>0: params_G += list(G.ups[-1].parameters())
if hasattr(G, "to_rgb"): params_G += list(G.to_rgb.parameters())
params_A = list(adapter.parameters())

OPT_D = torch.optim.Adam(params_D, lr=1e-4, betas=(0.0, 0.99))
OPT_G = torch.optim.Adam(params_G, lr=2e-4, betas=(0.0, 0.99))
OPT_A = torch.optim.Adam(params_A, lr=5e-4, betas=(0.0, 0.99))

from torch.optim.lr_scheduler import CosineAnnealingLR
sched_D = CosineAnnealingLR(OPT_D, T_max=STEPS_LITE, eta_min=1e-5)
sched_G = CosineAnnealingLR(OPT_G, T_max=STEPS_LITE, eta_min=1e-5)
sched_A = CosineAnnealingLR(OPT_A, T_max=STEPS_LITE, eta_min=1e-5)


from torch.cuda.amp import autocast, GradScaler
scaler_D = GradScaler(enabled=USE_AMP)
scaler_G = GradScaler(enabled=USE_AMP)

# z fixe pour suivi visuel
torch.manual_seed(1234)
FIX_PER_CLASS = 8
z_fixed = torch.randn(FIX_PER_CLASS, Z_DIM, device=DEVICE)

# --------------------------
# 6) Boucle d'entraînement “Lite”
# --------------------------
global_step, t0 = 0, time.time()
while global_step < STEPS_LITE:
    for x_real, y_lbl, _ in train_dl_gan:
        if global_step >= STEPS_LITE: break
        x_real = x_real.to(DEVICE); y_lbl = y_lbl.to(DEVICE)

        with torch.no_grad():
            c_cond = cond_from_proto(y_lbl, noise_std=0.0)  # (B, C_DIM)

        # === D
        OPT_D.zero_grad(set_to_none=True)
        with autocast(enabled=USE_AMP):
            d_real = Dc(x_real, c_cond)
            z = torch.randn(x_real.size(0), Z_DIM, device=DEVICE)
            x_fake = Gc(z, c_cond).detach()
            d_fake = Dc(x_fake, c_cond)
            loss_D = d_logistic_loss(d_real, d_fake)
        scaler_D.scale(loss_D).backward()
        scaler_D.unscale_(OPT_D); nn.utils.clip_grad_norm_(params_D, CLIP_D)
        scaler_D.step(OPT_D); scaler_D.update()
        log_metrics(global_step, d_loss=loss_D.item(),
                    ada_p=(ADA_STATE.get("p") if 'ADA_STATE' in globals() else None),
                    ada_acc=(ADA_STATE.get("acc_ema") if 'ADA_STATE' in globals() else None),
                    lrD=OPT_D.param_groups[0]['lr'])
        flush_history()

        # R1 périodique
        if (global_step % R1_EVERY) == 0:
            OPT_D.zero_grad(set_to_none=True)
            xr = x_real.detach().requires_grad_(True)
            with autocast(enabled=USE_AMP):
                drr = Dc(xr, c_cond)
                grad = torch.autograd.grad(drr.sum(), xr, create_graph=True)[0]
                r1  = 0.5 * R1_GAMMA * grad.pow(2).view(grad.size(0), -1).sum(1).mean()
            scaler_D.scale(r1).backward()
            scaler_D.unscale_(OPT_D); nn.utils.clip_grad_norm_(params_D, CLIP_D)
            scaler_D.step(OPT_D); scaler_D.update()
            log_metrics(global_step, r1=r1.item() if 'r1' in locals() else None)
            flush_history()


        if 'ada_update' in globals():
            with torch.no_grad(): ada_update(d_real.detach())

        # === G + Adapter
        OPT_G.zero_grad(set_to_none=True); OPT_A.zero_grad(set_to_none=True)
        with autocast(enabled=USE_AMP):
            e_batch = torch.stack([PROTOS[int(k.item())].to(DEVICE) for k in y_lbl], 0)
            c_train = adapter(e_batch)
            z = torch.randn(x_real.size(0), Z_DIM, device=DEVICE)
            xf = Gc(z, c_train)
            dfg = Dc(xf, c_train)
            loss_G = g_nonsat_loss(dfg)
        scaler_G.scale(loss_G).backward()
        scaler_G.unscale_(OPT_G); scaler_G.unscale_(OPT_A)
        nn.utils.clip_grad_norm_(params_G, CLIP_G); nn.utils.clip_grad_norm_(params_A, CLIP_G)
        scaler_G.step(OPT_G); scaler_G.step(OPT_A); scaler_G.update()
        # Step des schedulers
        sched_D.step(); sched_G.step(); sched_A.step()
        log_metrics(global_step, g_loss=loss_G.item(),
                    lrG=OPT_G.param_groups[0]['lr'], lrA=OPT_A.param_groups[0]['lr'])
        flush_history()


        # logs & samples
        if (global_step % LOG_EVERY) == 0:
            print(f"[{global_step:05d}] D={loss_D.item():.3f} G={loss_G.item():.3f} | {(time.time()-t0)/60:.1f} min")

        if (global_step % SAMPLE_EVERY) == 0:
            with torch.no_grad():
                rows = []
                for k in sorted(PROTOS.keys()):
                    labels = torch.full((FIX_PER_CLASS,), int(k), dtype=torch.long, device=DEVICE)
                    c_s = cond_from_proto(labels, noise_std=0.0)
                    imgs = Gc(z_fixed, c_s).clamp(-1,1)
                    rows.append(imgs)
                grid = torch.cat(rows, 0)
                out_path = SAMPLES_DIR / f"pixcell_lite_fix_step{global_step:06d}.png"
                save_image((grid+1)/2, out_path, nrow=FIX_PER_CLASS)
                print(f"🖼️ samples → {out_path}")

        if (global_step % 2000) == 0 and global_step > 0:
            ckpt = {
                "step": global_step,
                "Gc_map": Gc.map.state_dict(),
                "Dc_hproj": Dc.hproj.state_dict(),
                "Dc_proj": Dc.proj.state_dict(),
                "G_tail": getattr(G, "to_rgb", nn.Identity()).state_dict() if hasattr(G,"to_rgb") else {},
                "D_fc":   getattr(D, "fc", nn.Identity()).state_dict() if hasattr(D,"fc") else {},
                "adapter": adapter.state_dict(),
                "optD": OPT_D.state_dict(), "optG": OPT_G.state_dict(), "optA": OPT_A.state_dict(),
                "schD": sched_D.state_dict(), "schG": sched_G.state_dict(), "schA": sched_A.state_dict(),
            }
            torch.save(ckpt, SAMPLES_DIR / f"pixcell_lite_step{global_step:06d}.pt")


        global_step += 1

print("✅ Entraînement Lite (stabilisé) terminé.")
flush_history(force=True)
plot_training()

/tmp/ipykernel_119/2979164981.py:88: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_D = GradScaler(enabled=USE_AMP)
/tmp/ipykernel_119/2979164981.py:89: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_G = GradScaler(enabled=USE_AMP)
/tmp/ipykernel_119/2979164981.py:110: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=USE_AMP):
/tmp/ipykernel_119/2979164981.py:129: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=USE_AMP):
/tmp/ipykernel_119/2979164981.py:145: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=USE_AMP):


[00000] D=1.381 G=0.708 | 0.0 min
🖼️ samples → /workspace/samples/pixcell_lite_fix_step000000.png
[00200] D=2.010 G=0.689 | 0.7 min
[00400] D=1.868 G=0.302 | 1.3 min
🖼️ samples → /workspace/samples/pixcell_lite_fix_step000500.png
[00600] D=1.860 G=0.796 | 2.0 min
[00800] D=1.783 G=0.353 | 2.7 min
[01000] D=1.553 G=0.403 | 3.4 min
🖼️ samples → /workspace/samples/pixcell_lite_fix_step001000.png


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.11/site-packages/tensorboard/summary/writer/event_file_writer.py", line 244, in run
    self._run()
  File "/opt/conda/lib/python3.11/site-packages/tensorboard/summary/writer/event_file_writer.py", line 275, in _run
    self._record_writer.write(data)
  File "/opt/conda/lib/python3.11/site-packages/tensorboard/summary/writer/record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "/opt/conda/lib/python3.11/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 775, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "/opt/conda/lib/python3.11/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 167, in append
    self._write(filename, file_content, "ab" if binary_mode else "a")
  File "/opt

OSError: [Errno 9] Bad file descriptor